# DATA20001 Deep Learning - Group Project
## Image project

**Due Thursday, December 13, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imread` from `matplotlib.pyplot`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- In the exercises we used e.g., `torchvision.datasets.MNIST` to handle the loading of the data in suitable batches. Here, you need to handle the dataloading yourself.  The easiest way is probably to create a custom `Dataset`. [See for example here for a tutorial](https://github.com/utkuozbulak/pytorch-custom-dataset-examples).

## Download the data

In [2]:
# Configuration items collected into one place
cfg = {
    # Training files directories
    'training_files_root_dir': 'train',
    'annotation_dir': 'annotations', # Sub-directory
    'images_dir': 'images', # Sub-directory
    # Other directories
    'checkpoint_dir': "./checkpoints/",
    'log_dir': "./logs/"
#COLAB USERS!!! Use these definitions instead
#    'checkpoint_dir': "/content/gdrive/My Drive/checkpoints/",
#    'log_dir': "/content/gdrive/My Drive/logs/"
  }

# Constants
constants = {
    # These are possible labels defined in the dataset
    'label_titles': ["baby", "bird", "car", "clouds", "dog", "female", "flower",
                "male", "night", "people", "portrait", "river", "sea", "tree"]
}


In [3]:
import os

# Check if data files are present and obtain them if missing
if not os.path.isdir(cfg['training_files_root_dir']):
    from torchvision.datasets.utils import download_url
    import zipfile

    # Left here as not used anywhere else
    dl_file = 'dl2018-image-proj.zip'
    dl_url = 'https://users.aalto.fi/mvsjober/misc/'

    zip_path = os.path.join(cfg['training_files_root_dir'], dl_file)
    # Download data file if needed
    if not os.path.isfile(zip_path):
        download_url(dl_url + dl_file, root=cfg['training_files_root_dir'], filename=dl_file, md5=None)

    # Extract data file
    with zipfile.ZipFile(zip_path) as zip_f:
        zip_f.extractall(cfg['training_files_root_dir'])
else:
    print('Data is available')

Data is available


The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Your stuff goes here ...

## Lets parse and take a look at the data

In [1]:
# Imports save of os, which is done earlier
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re
from skimage import io, transform
import sklearn.metrics as metrics
import time
import torch
import torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset


In [4]:
def create_label_matrix(annotations_dir_name):
    # Create labels for use in dataset
    labels = np.zeros((20000,14))
    
    anno_dir = os.fsencode(annotations_dir_name)
    
    for idx, file in enumerate(sorted(os.listdir(anno_dir))):
        filename = os.fsdecode(file)
        with open(os.path.join(annotations_dir_name, filename)) as f:
            lines = f.readlines()
            for line in lines:
                # Image name indexing starts from 1, so convert
                # it to matrix index by subtracting one
                img_idx = int(line)-1
                labels[img_idx, idx] = 1

    return labels


In [5]:
# Parse label data
labs = create_label_matrix(os.path.join(cfg['training_files_root_dir'],cfg['annotation_dir']))

In [ ]:
# Label distributions
lab_counts = labs.sum(axis=0)
total_labs = lab_counts.sum()
sorted_indices = np.argsort(lab_counts)
labeless = (labs.sum(axis=1)==0).sum()


pad = 20
print("Class statistics\n")
print(f"{'Class':<{pad}}{'Count':<{pad}}{'% of all labels':<{pad}}{'% of all samples':<{pad}}")
for i in sorted_indices:
    print(f"{constants['label_titles'][i]:<{pad}}{int(lab_counts[i]):<{pad}}"+
          f"{lab_counts[i]/total_labs*100:.3}%{lab_counts[i]/len(labs)*100:16.3}%")
print(f"\n{'Total labels':<{pad}}{int(total_labs)}\n")
print(f"{'Samples with labels':<22} {len(labs)-labeless} ({(len(labs)-labeless)/len(labs)*100:.3}%)")
print(f"{'Samples without labels':<22} {labeless} ({labeless/len(labs)*100:.3}%)")


In [ ]:
# How many labels per sample
bins = np.unique(labs.sum(axis=1))
print(f"Sample can have this many labels: {bins}")
counts = np.array([(labs.sum(axis=1)==u).sum() for u in bins])
print(f"Sample counts by label counts from 0 to 5: {counts}")
plt.figure()
plt.bar(bins,counts)
plt.grid()
plt.title("Sample counts by label counts")
plt.show()

In [ ]:
# Let's take a look at the pictures

# Prints pictures with exactly c labels
def picture_by_label_count(c,size=5):
    inds = np.nonzero((labs.sum(axis=1) == c).astype(int))
    inds = np.random.choice(inds[0], size=size, replace=False)
    for i in inds:
        image = Image.open(os.path.join(cfg['training_files_root_dir'],
                                        cfg['images_dir'], f"im{i+1}.jpg"))
        imlabs = [constants['label_titles'][idx] for idx,l in enumerate(labs[i,:]) if l==1]
        plt.imshow(image,label=imlabs)
        plt.title(imlabs)
        plt.show()
    return np.nonzero(inds)

# Prints pictures that have label at index c (baby=0,..., tree=13)
def picture_by_label_class(c,size=5):
    inds = np.nonzero((labs[:,c] == 1).astype(int))
    inds = np.random.choice(inds[0], size=size, replace=False)
    for i in inds:
        image = Image.open(os.path.join(cfg['training_files_root_dir'],
                                        cfg['images_dir'], f"im{i+1}.jpg"))
        imlabs = [constants['label_titles'][idx] for idx,l in enumerate(labs[i,:]) if l==1]
        plt.imshow(image,label=imlabs)
        plt.title(imlabs)
        plt.show()
    return np.nonzero(inds)

# Pictures with different amount of labels
for i in range(6):
    print(f"Pictures with {i} label{'s' if i != 1 else ''}")
    picture_by_label_count(i,3)

In [ ]:
# Pictures with different classes
for i in range(14):
    print(f"Pictures with label {constants['label_titles'][i]}")
    picture_by_label_class(i,2)

In [ ]:
# Label relations
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder


graph_builder = LabelCooccurrenceGraphBuilder(weighted=True, include_self_edges=False)

edge_map = graph_builder.transform(labs)
print("{} labels, {} edges".format(len(constants['label_titles']), len(edge_map)))
print(edge_map)

In [ ]:
print("Top 10 most common label co-occurences\n")
for idx,pair in enumerate(sorted(edge_map,key=edge_map.get, reverse=True)):
    if idx==10:
        break
    print(f"({constants['label_titles'][pair[0]]}, {constants['label_titles'][pair[1]]}): \t{int(edge_map[pair])}")

In [ ]:
import matplotlib
occurence_matrix = np.zeros((14,14))
for e in edge_map:
    occurence_matrix[e[0],e[1]] = edge_map[e]
    occurence_matrix[e[1],e[0]] = edge_map[e]

plt.imshow(occurence_matrix,cmap="plasma")

## Data manipulation

In [ ]:
# Dataset class for easy access to data
class ImageDataset(Dataset):
    def __init__(self, index_map, transformations, labels, root_dir):
        self.index_map = index_map # Contenst in tuple (image number zero based, index to transformations array)
        self.labels = labels
        self.transformations = transformations # Array of transformations
        self.root_dir = root_dir

    def __getitem__(self, index):
        # Separated image number and transformation index
        img_idx, trans_idx = self.index_map[index]
        # Image filenames start with 1
        filename = f"im{img_idx + 1}.jpg"
        # Read the image in PIL format for it to work with transformations
        image = Image.open(os.path.join(self.root_dir, filename))
        imglabels = self.labels[img_idx]
        # Apply necessary transformations
        image = self.transformations[trans_idx](image)
        
        # This takes the V value from HSV transformation i.e. does color to grayscale transformation
        image = torch.max(image, dim=0)[0].unsqueeze(dim=0)

        return (image, imglabels)

    def __len__(self):
        return len(self.index_map)


In [ ]:
# Combine image and transformation indices into combined array, so that all different
# transformations are done to image
def combine_index_and_transform(indices, trnsfrms):
    cidx = []
    for tr in range(len(trnsfrms)):
        for idx in indices:
            cidx.append((idx, tr))
    return cidx


In [ ]:
# Storage for various sets related items
class TVTClass:
    size = 0 # Set size
    indices = [] # Image indices for this set
    transforms = 0 # Array of transformations for these images
    index_transform_map = [] # Tuple array of combine_index_and_transform result
    img_set = 0 # Dataset class instace
    loader = 0 # Dataloader class instance


In [ ]:
def plain_datasets(ctrain, cvalidation, labels, train_split):
    # Create array of indices and split them into two sets - training and validation
    # NOTE: This is simple way to include all images. To change that e.g. for downsampling
    # or upsampling TVTClass.indices (the last line below where those two indices are set)
    # is the key, so changing these indices creation will achieve wanted result.
    image_indices = list(range(len(labels)))
    ctrain.size = int(train_split * len(image_indices))
    cvalidation.size = len(image_indices) - ctrain.size
    # Randomize what goes into what set
    all_indices = np.random.permutation(image_indices)
    ctrain.indices, cvalidation.indices = all_indices[:ctrain.size], all_indices[ctrain.size:]

    # Here are the transforms that are applied to the images
    # Images must be transformed at least to tensor, grayscale conversion is done inside dataset class

    # Training data related transformations
    ctrain.transforms = [
        transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
        transforms.Compose([transforms.Grayscale(num_output_channels=1), transforms.RandomHorizontalFlip(p=0.50),
                            transforms.RandomRotation(degrees=15, resample=False, expand=False, center=None),
                            transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])]
    ctrain.index_transform_map = combine_index_and_transform(ctrain.indices, ctrain.transforms)

    # Validation data related transformations
    # Typically nothing else than to tensor
    cvalidation.transforms = [
        transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])]
    cvalidation.index_transform_map = combine_index_and_transform(cvalidation.indices, cvalidation.transforms)


In [ ]:
# labs = return value of create_label_matrix function
# sample_size = number of training samples in each set
# train_split = percentage of samples to be assigned to training set
# labs_indices = list of label indices
def create_crafted_datasets(labs, sample_size, train_split, labs_indices, ctrain, cvalidation):
    labssets = {}
    valset = {}
    trainset = {}
    # Split available images, so that some of each label group are in validation set
    for i in labs_indices:
        # Select images based on one label. Obtain row numbers, which are directly translatable to images,
        # for which particular label is present.
        labssets[i] = set(np.where(labs[:,i]==1)[0])
        # Calculate amount to keep in validation set
        valset_size = int((1 - train_split) * len(labssets[i]))
        # Select randomly validation set images
        valset[i] = set(np.random.choice(list(labssets[i]), valset_size, False))
        # Put rest in training set
        trainset[i] = labssets[i] - valset[i]

    # Handle images without any labels similarly to above and add them to same sets as last
    nolab_idx = len(labs_indices)
    # Obtain row numbers for those rows that have no ones in any column
    labssets[nolab_idx] = set(np.where(~labs.any(axis=1))[0])
    valset_size = int((1 - train_split) * len(labssets[nolab_idx]))
    valset[nolab_idx] = set(np.random.choice(list(labssets[nolab_idx]), valset_size, False))
    trainset[nolab_idx] = labssets[nolab_idx] - valset[nolab_idx]
    # Append no label index to send of other indices
    labs_indices = np.append(labs_indices, nolab_idx)

    # Create training groups of equal size by down- or upsampling as necessary
    sampled_labs = {}
    for i in labs_indices:
        # If training set size is equal or greater than requested sample size
        # do not utilize same image twice
        if len(trainset[i]) >= sample_size:
            repl = False
        else:
            repl = True
        sampled_labs[i] = np.random.choice(list(trainset[i]), sample_size, repl)
        # Add extra images not used in training to validation set
        # Note: this might be a place to tune down e.g. amount of images without labels
        if not repl:
            valset[i] = valset[i] | (trainset[i] - set(sampled_labs[i]))

    # Following can be used if no transformation is wanted to some images as noted in
    # code that is in comments below
    # normtransforms = [transforms.Compose([transforms.Grayscale(num_output_channels=1),
    #                                       transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])]
    # Training set random transformations
    rndtransforms = [
        transforms.Compose([transforms.RandomHorizontalFlip(p=0.60),
                            transforms.RandomRotation(degrees=45, resample=False, expand=False, center=None),
                            transforms.RandomResizedCrop(128, scale=(0.70, 1.0), ratio=(0.75, 1.3333333333333333)),
                            transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])]
    # Validation set transforms
    valtransforms = [transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])]

    # Create combined lists of images and transformations for both datasets
    train_indices = []
    train_indices_combo = []
    validation_indices = []
    validation_indices_combo = []
    for i in labs_indices:
        # Can be used to omit transformation e.g. in case training set is adequate size
        # if len(trainset[i]) >= sample_size:
        #     trnsfrm = normtransforms
        # else:
        trnsfrm = rndtransforms
        # Training set handling
        tidx_trnsfrm = combine_index_and_transform(sampled_labs[i], trnsfrm)
        train_indices = np.append(train_indices, sampled_labs[i])
        train_indices_combo += tidx_trnsfrm
        # Validation set handling
        vidx_trnsfrm = combine_index_and_transform(valset[i], valtransforms)
        validation_indices += valset[i]
        validation_indices_combo += vidx_trnsfrm

    # Fill in the settings for training and validation
    ctrain.size = len(train_indices)
    ctrain.indices = train_indices
    ctrain.transforms = rndtransforms
    ctrain.index_transform_map = train_indices_combo
    cvalidation.size = len(validation_indices)
    cvalidation.indices = validation_indices
    cvalidation.transforms = valtransforms
    cvalidation.index_transform_map = validation_indices_combo


In [ ]:
# Here we create iterators for training and validation datasets

# BATCH_SIZE is obvious
# TRAIN_SPLIT tells how big portion of data stays in the training set
# TRAIN_SAMPLE_SIZE indicates size of each group of labels in training set
BATCH_SIZE = 32
TRAIN_SPLIT = 0.75
TRAIN_SAMPLE_SIZE = 3000

labels = create_label_matrix(os.path.join(cfg['training_files_root_dir'],cfg['annotation_dir']))

ctrain = TVTClass()
cvalidation = TVTClass()

# The basic dataset usage
#plain_datasets(ctrain, cvalidation, labels, TRAIN_SPLIT)

# Following helps in debugging whereas all is needed is just list of indices for labels
sorted_indices = np.argsort(labels.sum(axis=0))
create_crafted_datasets(labels, TRAIN_SAMPLE_SIZE, TRAIN_SPLIT, sorted_indices, ctrain, cvalidation)

# Create datasets for training and validation
ctrain.img_set = ImageDataset(ctrain.index_transform_map, ctrain.transforms, labels,
                             os.path.join(cfg['training_files_root_dir'],cfg['images_dir']))
cvalidation.img_set = ImageDataset(cvalidation.index_transform_map, cvalidation.transforms, labels,
                                  os.path.join(cfg['training_files_root_dir'],cfg['images_dir']))

# Create dataloaders for training and validation
ctrain.loader = torch.utils.data.DataLoader(dataset=ctrain.img_set, batch_size=BATCH_SIZE,
                                           shuffle=True)
cvalidation.loader = torch.utils.data.DataLoader(dataset=cvalidation.img_set,batch_size=BATCH_SIZE,
                                                shuffle=False)


In [ ]:
# Check out what one batch has
# Useful for checking out the effects of transfomations
for idx, (image,label) in enumerate(ctrain.loader):
    for i in range(image.shape[0]):
        s = []
        for j in range(len(label[0])):
            if int(label[i,j]) == 1:
                s.append(constants['label_titles'][j])
        plt.title(s)
        plt.imshow(image[i].squeeze(),cmap="gray",label=s)
        plt.show()
    break

In [ ]:
# Check out what one batch has
# Useful for checking out the effects of transfomations
for idx, (image,label) in enumerate(cvalidation.loader):
    for i in range(image.shape[0]):
        s = []
        for j in range(len(label[0])):
            if int(label[i,j]) == 1:
                s.append(constants['label_titles'][j])
        plt.title(s)
        plt.imshow(image[i].squeeze(),cmap="gray",label=s)
        plt.show()
    break

## Functions for defining and training the model

In [ ]:
"""
Multilabeling images

Template for the network training procedure, which includes logging in the middle of training epochs,
logging each epoch losses and accuracies to a separate file, and saving network and optimizer parameters
to a separate file.
"""

#Hardware detection
if torch.cuda.is_available():
    print('Using GPU.')
    device = torch.device('cuda')
else:
    print('Using CPU.')
    device = torch.device('cpu')

def batch_accuracy(preds, y):
    """OBSOLETE!!!
    Count accuracy for given batch
    Parameters:
       preds - values predicted by the model
       y - target values
    Returns:
       Amount of correct answers normalized ie value between 0.0 and 1.0
    """
    
    # Need to copy to CPU when using GPU
    pred = preds.cpu()
    target = y.cpu()
    
    # Naive thresholding, if value at least 0.5 label is predicted as true
    pred = pred.round()
    
    # Exact Match Ratio
    # Prediction is correct only if all labels for the sample are correct
    return (pred.eq(target).sum(dim=1)==14).float().mean()

def threshold(z):
    return np.round(z)

def train(model, iterator, optimizer, criterion, logPerInterval=0.25):
    """Training method
    Parameters:
        model - Used neural net
        iterator - Iterator for training data
        optimizer - Used optimizer
        criterion - Used loss function
        logPerInterval - Fraction of epoch at which time intermediate logs are printed to output,
                         in case one epoch takes too long. Set to zero to make no such logs.
    Returns:
        Training loss
        Training accuracy
    """
    
    epoch_loss = 0
    model.train()
    preds = np.zeros((len(iterator)*iterator.batch_size,14))
    targets = np.zeros((len(iterator)*iterator.batch_size,14))
    prev = 0
    for count, (data, target) in enumerate(iterator):
        data = data.to(device)
        target = target.float().to(device)
        
        optimizer.zero_grad()
        predictions = model(data)
        
        loss = criterion(predictions, target)
        #acc = batch_accuracy(predictions, target)
        cur_end = len(predictions) + prev
        preds[prev:cur_end,:] = predictions.cpu().detach().numpy()
        targets[prev:cur_end,:] = target
        prev = cur_end
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
            
        progress = count / len(iterator)
        if logPerInterval > 0 and count > 0 and count % (int(len(iterator) * logPerInterval)) == 0:
            print(f'    {progress*100:.2f}%: Loss: {epoch_loss/count:.3f}')
    
    epoch_acc = metrics.f1_score(targets,threshold(preds),average="micro")
    return epoch_loss / len(iterator), epoch_acc

def evaluate(model, iterator, criterion, return_preds=False):
    """Evaluation method, used for both validation and test sets
    Parameters:
        model - Used neural net
        iterator - Iterator for training data
        criterion - Used loss function
    Returns:
        Loss
        Accuracy
    """

    epoch_loss = 0
    model.eval()
    preds = np.zeros((len(iterator)*iterator.batch_size,14))
    targets = np.zeros((len(iterator)*iterator.batch_size,14))
    prev = 0
    with torch.no_grad():
        for idx, (data,target) in enumerate(iterator,1):
            data = data.to(device)
            target = target.float().to(device)
            predictions = model(data)
            
            # Save the predictions and true labels
            # for further use
            cur_end = len(predictions) + prev
            preds[prev:cur_end,:] = predictions
            targets[prev:cur_end,:] = target
            prev = cur_end
            
            
            loss = criterion(predictions, target)
            #acc = batch_accuracy(predictions, target)
            epoch_loss += loss.item()
            #epoch_acc += acc.item()
            
    epoch_acc = metrics.f1_score(targets,threshold(preds),average="micro")
    if return_preds:
        return epoch_loss / len(iterator), epoch_acc, (preds, target)
    return epoch_loss / len(iterator), epoch_acc

def saveCheckpoint(epoch, bestScore, model, optimizer, filename):
    """Saves checkpoint to external file
    Parameters:
        epoch - Current epoch
        bestScore - Best validation set f1 score obtained in training
        model - Neural net, whose parameters are saved
        optimizer - Optimizer state to save
        filename - Name of the checkpoint file
    """
    
    states = {
        'epoch': epoch + 1,
        'bestScore': bestScore,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }
    torch.save(states, filename)

In [ ]:
# Baseline
# Name: 3conv_1fc
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        self.input_ch = 1
        self.conv1_out = 64
        self.conv2_out = 128
        self.conv3_out = 256
        
        self.fc1_in = 256*9*9
        self.fc1_out = 1000
        
        self.fc2_out = 1000
        
        
        # Trivial CNN just for testing that everything works
        self.conv1 = nn.Conv2d(self.input_ch,self.conv1_out,kernel_size=3,stride=1)
        self.bn1 = nn.BatchNorm2d(self.conv1_out)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
          
        self.conv2 = nn.Conv2d(self.conv1_out,self.conv2_out,kernel_size=3,stride=1)
        self.bn2 = nn.BatchNorm2d(self.conv2_out)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv3 = nn.Conv2d(self.conv2_out,self.conv3_out,kernel_size=2,stride=1)
        self.bn3 = nn.BatchNorm2d(self.conv3_out)
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(self.fc1_in, self.fc1_out)
        self.bn4 = nn.BatchNorm1d(self.fc1_out)
        self.do1 = nn.Dropout(dropout)
        
        # 14 classes
        self.last = nn.Linear(self.fc1_out, 14)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        x = x.view(-1, self.fc1_in)
        x = self.do1(torch.relu(self.bn4(self.fc1(x))))
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

In [ ]:
# Loads a model
# Needs a model of same structure as parameter
def load_model(model, optimizer, filePrefix, timestampToLoad, checkpointPath = "./checkpoints/"):  
    checkPoint = torch.load("{}{}-{}.cpt".format(checkpointPath, filePrefix, timestampToLoad))
    epoch = checkPoint['epoch']
    bestScore = checkPoint['bestScore']
    model.load_state_dict(checkPoint['model'])
    optimizer.load_state_dict(checkPoint['optimizer'])
    # Following needs to be reconsidered if not using GPU (comment out maybe?)
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()    
    
    return model, optimizer, epoch, bestScore

# Initialise model if training from scratch and not loading it from file
def init_model(dropout=0.5):
    model = Net(dropout = dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    return model, optimizer, 0, -1.0

def show_plots(train_losses, valid_losses, train_score, valid_score, n_epochs, startEpoch=0):
    #Draw plots (only plots current session, there's another script to plot what was written to log)
    plt.plot(np.arange(startEpoch+1,startEpoch + n_epochs+1), train_losses, label='Training loss')
    plt.plot(np.arange(startEpoch+1,startEpoch + n_epochs+1), valid_losses, label='Validation loss')
    plt.title('Losses')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

    plt.plot(np.arange(startEpoch+1,startEpoch + n_epochs+1), train_score, label='Training F1 score')
    plt.plot(np.arange(startEpoch+1,startEpoch + n_epochs+1), valid_score, label='Validation F1 score')
    plt.title('F1 score')
    plt.xlabel('Epochs')
    plt.ylabel('F1 score')
    plt.legend()
    plt.grid()
    plt.show()

# Used to do both training and evaluation of the model
# If save_results is False, doesn't save logs or checkpoints, mainly for Colab
def train_and_evaluate(checkpoint, criterion, train_loader, validation_loader, n_epochs, startEpoch=0,
                       save_results=True,  saveInterval = 1, checkPointRotation = 3,
                       recentFilePrefix="net01", bestFilePrefix="best01"):
    
    criterion = criterion.to(device)
    model = checkpoint[0]
    optimizer = checkpoint[1]
    startEpoch = checkpoint[2]
    best_score = checkpoint[3]

    recentSaveCount = 0
    bestSaveCount = 0
    # Lock in starting datetime as part of filename for this session
    timestampToSave = "{}".format(time.strftime("%Y%m%d-%H%M%S"))

    train_losses = []
    valid_losses = []
    train_score = []
    valid_score = []

    for epoch in range(startEpoch, startEpoch + n_epochs):
        train_loss, train_acc = train(model, train_loader, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, validation_loader, criterion)
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_score.append(train_acc)
        valid_score.append(valid_acc)
        print(f"| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | "+
              f"Train Acc: {train_acc:.3f} | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc:.3f} |")
        # Log loss and accuracy in case we want to draw graphs
        # Each session creates new log file. If you want to append to existing log file,
        # just remove the timestamp part from the end of filename.
        if save_results:
            with open("{}{}-{}.txt".format(cfg['log_dir'], recentFilePrefix, timestampToSave), 'a') as logFile:
                logFile.write("{} {} {} {} {}\n".format(epoch+1, train_loss, train_acc, valid_loss, valid_acc))

            #Save recent model
            if epoch % saveInterval == 0:
                saveCheckpoint(epoch, best_score, model, optimizer, "{}{}-{}-{}.cpt".format(
                    cfg['checkpoint_dir'], recentFilePrefix, timestampToSave, recentSaveCount % checkPointRotation))
                recentSaveCount += 1
            #Save best model
            if valid_acc > best_score:
                saveCheckpoint(epoch, best_score, model, optimizer, "{}{}-{}-{}.cpt".format(
                    cfg['checkpoint_dir'], bestFilePrefix, timestampToSave, bestSaveCount % checkPointRotation))
                best_score = valid_acc
                bestSaveCount += 1


    #Save one last time at the end of execution
    if save_results:
        saveCheckpoint(epoch, best_score, model, optimizer, 
                       "{}{}-{}-{}.cpt".format(cfg['checkpoint_dir'], recentFilePrefix, timestampToSave, recentSaveCount % checkPointRotation))    
    show_plots(train_losses, valid_losses, train_score, valid_score, n_epochs, startEpoch)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%%time
# 1. Create dataset and loaders
#
# 2. Create or load the network
#
# 3. Train and evaluate it

#Obviously savepaths would be different among us developers,
#but I suggest timestamping and naming models to preserve backups
#CHKPNT_PTH = "./checkpoints/"
#LOG_PTH = "./logs/"

# COLAB USERS!! Uncomment this to mount gdrive directory to save logs and checkpoints
#from google.colab import drive
#drive.mount('/content/gdrive')

N_EPOCHS = 35

#train_loader = train_loader
#validation_loader = validation_loader

params = [0.5]

for idx,param in enumerate(params):
    filePrefix = f"2deepconv_1fc"
    bestPrefix = f"2deepconv_1fc_best"
    criterion = nn.BCELoss()

    train_and_evaluate(init_model(), criterion, ctrain.loader, cvalidation.loader, N_EPOCHS, recentFilePrefix=filePrefix,
                      bestFilePrefix=bestPrefix)


In [ ]:
"""
Load some saved model (has to have same architecture as the Net class defined above)
and evaluate with it one batch from validation set. Show the pictures and corresponding
target labels and labels predicted by the model.
"""
filePrefix = "best01"
timestampToLoad = "20181216-182441-1"
model = Net(0.5)
checkPoint = torch.load("{}{}-{}.cpt".format(cfg['checkpoint_dir'], filePrefix, timestampToLoad))
model.load_state_dict(checkPoint['model'])
model = model.to(device)
with torch.no_grad():
    for idx, (data,target) in enumerate(cvalidation.loader,1):
        data = data.to(device)
        target = target.float().to(device)
        predictions = model(data)
        for idx, img in enumerate(data):
            realLabels = []
            for i in range(14):
                if target[idx, i] == 1.0:
                    realLabels.append(constants['label_titles'][i])
            predLabels = []
            for i in range(14):
                if predictions[idx, i] > 0.5:
                    predLabels.append(constants['label_titles'][i])            
            plt.imshow(img.squeeze(0))
            plt.title('real labels: {} -> pred labels: {}'.format(realLabels, predLabels))
            plt.show()
        break

## Different network configs that were tried

In [ ]:
# Name: 4conv_1fc
# new best with 0.708
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        self.input_ch = 1
        self.conv1_out = 64
        self.conv2_out = 128
        self.conv3_out = 256
        self.conv4_out = 512
        
        self.fc1_in = 512*8*8
        self.fc1_out = 1000
        
        self.fc2_out = 1000
        
        
        # Trivial CNN just for testing that everything works
        self.conv1 = nn.Conv2d(self.input_ch,self.conv1_out,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(self.conv1_out)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2)
          
        self.conv2 = nn.Conv2d(self.conv1_out,self.conv2_out,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(self.conv2_out)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv3 = nn.Conv2d(self.conv2_out,self.conv3_out,kernel_size=2,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(self.conv3_out)
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv4 = nn.Conv2d(self.conv3_out,self.conv4_out,kernel_size=2,stride=1,padding=1)
        self.bn4 = nn.BatchNorm2d(self.conv4_out)
        self.pool4 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(self.fc1_in, self.fc1_out)
        self.bn5 = nn.BatchNorm1d(self.fc1_out)
        self.do1 = nn.Dropout(dropout)
        
        # 14 classes
        self.last = nn.Linear(self.fc1_out, 14)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = torch.relu(self.bn4(self.conv4(x)))
        x = self.pool4(x)
        
        x = x.view(-1, self.fc1_in)
        x = self.do1(torch.relu(self.bn5(self.fc1(x))))
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

In [ ]:

# Name: 2deepconv_1fc
# best 0.713 but is extremely slow to train
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        self.input_ch = 1
        
        self.conv1_out = 64
        self.conv2_out = 128
        self.conv3_out = 256
        
        self.conv4_out = 128
        self.conv5_out = 256
        self.conv6_out = 256
        
        self.conv7_out = 128
        
        
        self.fc1_in = 128*12*12
        self.fc1_out = 1000
        
        self.fc2_out = 1000
        
        
        # Lets try stacked convolutions
        self.conv1 = nn.Conv2d(self.input_ch,self.conv1_out,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(self.conv1_out)
          
        self.conv2 = nn.Conv2d(self.conv1_out,self.conv2_out,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(self.conv2_out)
        
        self.conv3 = nn.Conv2d(self.conv2_out,self.conv3_out,kernel_size=3,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(self.conv3_out)
        
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
        
        # Dimensionality reduction via 1D convolution
        self.conv4 = nn.Conv2d(self.conv3_out,self.conv4_out,kernel_size=1,stride=1)
        self.bn4 = nn.BatchNorm2d(self.conv4_out)
        
        self.conv5 = nn.Conv2d(self.conv4_out,self.conv5_out,kernel_size=3,stride=1)
        self.bn5 = nn.BatchNorm2d(self.conv5_out)
        
        self.conv6 = nn.Conv2d(self.conv5_out,self.conv6_out,kernel_size=3,stride=1)
        self.bn6 = nn.BatchNorm2d(self.conv6_out)
        
        self.pool2 = nn.MaxPool2d(kernel_size=3,stride=3)
        
        # Dimensionality reduction via 1D convolution
        self.conv7 = nn.Conv2d(self.conv6_out,self.conv7_out,kernel_size=1,stride=1)
        self.bn7 = nn.BatchNorm2d(self.conv7_out)
        
        self.fc1 = nn.Linear(self.fc1_in, self.fc1_out)
        self.bn8 = nn.BatchNorm1d(self.fc1_out)
        self.do1 = nn.Dropout(dropout)
        
        # 14 classes
        self.last = nn.Linear(self.fc1_out, 14)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn4(self.conv4(x)))
        x = torch.relu(self.bn5(self.conv5(x)))
        x = torch.relu(self.bn6(self.conv6(x)))
        x = self.pool2(x)
        x = torch.relu(self.bn7(self.conv7(x)))
        
        x = x.view(-1, self.fc1_in)
        x = self.do1(torch.relu(self.bn8(self.fc1(x))))
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

In [ ]:
# Name: 3conv_1fc_pad_v1
# score 0.694 but didn't converge after 30 epochs
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        self.input_ch = 1
        self.conv1_out = 128
        self.conv2_out = 256
        self.conv3_out = 512
        
        self.fc1_in = 512*11*11
        self.fc1_out = 1000
        
        self.fc2_out = 1000
        
        
        # Trivial CNN just for testing that everything works
        self.conv1 = nn.Conv2d(self.input_ch,self.conv1_out,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(self.conv1_out)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
          
        self.conv2 = nn.Conv2d(self.conv1_out,self.conv2_out,kernel_size=3,stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(self.conv2_out)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv3 = nn.Conv2d(self.conv2_out,self.conv3_out,kernel_size=2,stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(self.conv3_out)
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(self.fc1_in, self.fc1_out)
        self.bn4 = nn.BatchNorm1d(self.fc1_out)
        self.do1 = nn.Dropout(dropout)
        
        # 14 classes
        self.last = nn.Linear(self.fc1_out, 14)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        x = x.view(-1, self.fc1_in)
        x = self.do1(torch.relu(self.bn4(self.fc1(x))))
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

In [ ]:
# trivial_net
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        # Trivial CNN just for testing that everything works
        self.conv1 = nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
        
        # 14 classes
        self.last = nn.Linear(16*42*42, 14)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = x.view(-1, 16*42*42)
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

In [ ]:
# Name: 3conv_2fc
# Used dropout of 0.6
class Net(nn.Module):
    """ Neural network used for this project """
    def __init__(self, dropout):  #TODO: Have more hyperparameters as arguments
        super().__init__()
        
        self.input_ch = 1
        self.conv1_out = 64
        self.conv2_out = 128
        self.conv3_out = 256
        
        self.fc1_in = 256*9*9
        self.fc1_out = 1000
        
        self.fc2_out = 1000
        
        
        # Trivial CNN just for testing that everything works
        self.conv1 = nn.Conv2d(self.input_ch,self.conv1_out,kernel_size=3,stride=1)
        self.bn1 = nn.BatchNorm2d(self.conv1_out)
        self.pool1 = nn.MaxPool2d(kernel_size=3,stride=3)
          
        self.conv2 = nn.Conv2d(self.conv1_out,self.conv2_out,kernel_size=3,stride=1)
        self.bn2 = nn.BatchNorm2d(self.conv2_out)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv3 = nn.Conv2d(self.conv2_out,self.conv3_out,kernel_size=2,stride=1)
        self.bn3 = nn.BatchNorm2d(self.conv3_out)
        self.pool3 = nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.fc1 = nn.Linear(self.fc1_in, self.fc1_out)
        self.bn4 = nn.BatchNorm1d(self.fc1_out)
        self.do1 = nn.Dropout(dropout)
        
        self.fc2 = nn.Linear(self.fc1_out, self.fc2_out)
        self.bn5 = nn.BatchNorm1d(self.fc2_out)
        self.do2 = nn.Dropout(dropout)
        
        # 14 classes
        self.last = nn.Linear(self.fc2_out, 14)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        x = x.view(-1, self.fc1_in)
        x = self.do1(torch.relu(self.bn4(self.fc1(x))))
        x = self.do2(torch.relu(self.bn5(self.fc2(x))))
        # Sigmoid to make all values between 0 and 1
        x = torch.sigmoid(self.last(x))
        return x

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
torch.save(model.state_dict(), 'model.pkl')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".  [See more here on how to load the model back](https://github.com/pytorch/pytorch/blob/761d6799beb3afa03657a71776412a2171ee7533/docs/source/notes/serialization.rst) if you want to continue training later.

## Download test set

The testset will be made available during the last week before the deadline and can be downloaded in the same way as the training set.

## Predict for test set

You should return your predictions for the test set in a plain text file.  The text file contains one row for each test set image.  Each row contains a binary prediction for each label (separated by a single space), 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
The order of the rows should be according to the numeric order of the image numbers.  In the test set, this means that the first row refers to image `im20001.jpg`, the second to `im20002.jpg`, and so on.

If you have the prediction output matrix prepared in `y` you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')